Imports

In [2]:
# Importando bibliotecas essenciais para análise de dados e machine learning
import pandas as pd  # Manipulação e análise de dados
import numpy as np  # Operações matemáticas e manipulação de arrays

# Visualização de dados
import matplotlib.pyplot as plt  # Criação de gráficos
import seaborn as sns  # Visualização estatística

# Pré-processamento de dados
from sklearn.model_selection import train_test_split  # Divisão de dados em treino e teste
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder  # Escalonamento e codificação

# Modelagem e avaliação
from sklearn.linear_model import LinearRegression, LogisticRegression  # Modelos de regressão
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor  # Modelos baseados em árvores
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, confusion_matrix  # Métricas de avaliação

from IPython.display import display


In [33]:
df_aeronave = pd.read_csv("aeronave.csv", encoding='latin1', sep=';')
df_fator = pd.read_csv("fator_contribuinte.csv", encoding='latin1', sep=';')
df_recomendacao = pd.read_csv("recomendacao.csv", encoding='latin1', sep=';')
df_tipo = pd.read_csv("ocorrencia_tipo.csv", encoding='latin1', sep=';')
df_ocorrencia = pd.read_csv("ocorrencia.csv", encoding='latin1', sep=';')

df_merged = df_ocorrencia.merge(df_aeronave, on='codigo_ocorrencia2', how='left')
df_merged = df_merged.merge(df_fator, on='codigo_ocorrencia3', how='left')
df_merged = df_merged.merge(df_recomendacao, on='codigo_ocorrencia4', how='left')
df_merged = df_merged.merge(df_tipo, on='codigo_ocorrencia1', how='left')

display("Colunas do DataFrame:")
display(df_merged.columns)

display("Tipos de dados de cada coluna:")
display(df_merged.dtypes)

display("Quantidade de valores nulos por coluna:")
df_merged.replace("***", np.nan, inplace=True)
display(df_merged.isnull().sum())
display("valores unicos por coluna:")
display(df_merged.nunique())
display(df_merged.head())

'Colunas do DataFrame:'

Index(['codigo_ocorrencia', 'codigo_ocorrencia1', 'codigo_ocorrencia2',
       'codigo_ocorrencia3', 'codigo_ocorrencia4', 'ocorrencia_classificacao',
       'ocorrencia_latitude', 'ocorrencia_longitude', 'ocorrencia_cidade',
       'ocorrencia_uf', 'ocorrencia_pais', 'ocorrencia_aerodromo',
       'ocorrencia_dia', 'ocorrencia_hora', 'investigacao_aeronave_liberada',
       'investigacao_status', 'divulgacao_relatorio_numero',
       'divulgacao_relatorio_publicado', 'divulgacao_dia_publicacao',
       'total_recomendacoes', 'total_aeronaves_envolvidas',
       'ocorrencia_saida_pista', 'aeronave_matricula',
       'aeronave_operador_categoria', 'aeronave_tipo_veiculo',
       'aeronave_fabricante', 'aeronave_modelo', 'aeronave_tipo_icao',
       'aeronave_motor_tipo', 'aeronave_motor_quantidade', 'aeronave_pmd',
       'aeronave_pmd_categoria', 'aeronave_assentos',
       'aeronave_ano_fabricacao', 'aeronave_pais_fabricante',
       'aeronave_pais_registro', 'aeronave_registro_catego

'Tipos de dados de cada coluna:'

codigo_ocorrencia                    int64
codigo_ocorrencia1                   int64
codigo_ocorrencia2                   int64
codigo_ocorrencia3                   int64
codigo_ocorrencia4                   int64
ocorrencia_classificacao            object
ocorrencia_latitude                 object
ocorrencia_longitude                object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_pais                     object
ocorrencia_aerodromo                object
ocorrencia_dia                      object
ocorrencia_hora                     object
investigacao_aeronave_liberada      object
investigacao_status                 object
divulgacao_relatorio_numero         object
divulgacao_relatorio_publicado      object
divulgacao_dia_publicacao           object
total_recomendacoes                  int64
total_aeronaves_envolvidas           int64
ocorrencia_saida_pista              object
aeronave_matricula                  object
aeronave_op

'Quantidade de valores nulos por coluna:'

codigo_ocorrencia                      0
codigo_ocorrencia1                     0
codigo_ocorrencia2                     0
codigo_ocorrencia3                     0
codigo_ocorrencia4                     0
ocorrencia_classificacao               0
ocorrencia_latitude                 7845
ocorrencia_longitude                7847
ocorrencia_cidade                      0
ocorrencia_uf                         22
ocorrencia_pais                        0
ocorrencia_aerodromo                   0
ocorrencia_dia                         0
ocorrencia_hora                        2
investigacao_aeronave_liberada         0
investigacao_status                    1
divulgacao_relatorio_numero        10122
divulgacao_relatorio_publicado         0
divulgacao_dia_publicacao          11158
total_recomendacoes                    0
total_aeronaves_envolvidas             0
ocorrencia_saida_pista                 0
aeronave_matricula                     1
aeronave_operador_categoria        34934
aeronave_tipo_ve

'valores unicos por coluna:'

codigo_ocorrencia                  13037
codigo_ocorrencia1                 13037
codigo_ocorrencia2                 13037
codigo_ocorrencia3                 13037
codigo_ocorrencia4                 13037
ocorrencia_classificacao               3
ocorrencia_latitude                 3699
ocorrencia_longitude                3706
ocorrencia_cidade                   1426
ocorrencia_uf                         27
ocorrencia_pais                        1
ocorrencia_aerodromo                 749
ocorrencia_dia                      5108
ocorrencia_hora                     1255
investigacao_aeronave_liberada         2
investigacao_status                    2
divulgacao_relatorio_numero         2881
divulgacao_relatorio_publicado         2
divulgacao_dia_publicacao            344
total_recomendacoes                   24
total_aeronaves_envolvidas             5
ocorrencia_saida_pista                 2
aeronave_matricula                  6781
aeronave_operador_categoria           11
aeronave_tipo_ve

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,recomendacao_dia_assinatura,recomendacao_dia_encaminhamento,recomendacao_dia_feedback,recomendacao_conteudo,recomendacao_status,recomendacao_destinatario_sigla,recomendacao_destinatario,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao
0,86954,86954,86954,86954,86954,ACIDENTE,-29.08555569444,-53.86555555555,TUPANCIRETï¿½,RS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COLISÃO COM OBSTÁCULO DURANTE A DECOLAGEM E POUSO,COLISÃO COM OBSTÁCULO DURANTE A DECOLAGEM E POUSO,CTOL
1,86953,86953,86953,86953,86953,ACIDENTE,-9.596944444444,-35.96972222222,PILAR,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FALHA OU MAU FUNCIONAMENTO DO MOTOR,FALHA OU MAU FUNCIONAMENTO DO MOTOR,SCF-PP
2,86952,86952,86952,86952,86952,INCIDENTE,-2.586944444444,-44.23611111111,Sï¿½O LUï¿½S,MA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
3,86951,86951,86951,86951,86951,INCIDENTE GRAVE,-27.13388888888,-52.65888888888,CHAPECï¿½,SC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EXCURSÃO DE PISTA,EXCURSÃO DE PISTA,RE
4,86950,86950,86950,86950,86950,INCIDENTE,-23.43555555555,-46.47305555555,GUARULHOS,SP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COLISÃO COM AVE,COLISÃO COM AVE,BIRD


Base de dados de aeronaves